## CRF解决NER任务


![jupyter](./imgs/crf.jpeg)

In [7]:

import warnings

import sklearn_crfsuite

import util

warnings.filterwarnings('ignore')

### 1. 读取数据

In [2]:
train_data_file = './bio/train.bio'
test_data_file = './bio/test.bio'

In [3]:
# 读取bio格式数据
train_sentences = util.read_bio_data(train_data_file)
test_sentences = util.read_bio_data(test_data_file)
len(train_sentences), len(test_sentences)

(10748, 1343)

In [4]:
test_sentences[0][:30]

[['彭', 'B-nr', 'B-name'],
 ['小', 'I-nr', 'I-name'],
 ['军', 'E-nr', 'I-name'],
 ['认', 'B-v', 'O'],
 ['为', 'E-v', 'O'],
 ['，', 'S-x', 'O'],
 ['国', 'B-s', 'O'],
 ['内', 'E-s', 'O'],
 ['银', 'B-n', 'O'],
 ['行', 'E-n', 'O'],
 ['现', 'B-t', 'O'],
 ['在', 'E-t', 'O'],
 ['走', 'S-v', 'O'],
 ['的', 'S-uj', 'O'],
 ['是', 'S-v', 'O'],
 ['台', 'B-ns', 'B-address'],
 ['湾', 'E-ns', 'I-address'],
 ['的', 'S-uj', 'O'],
 ['发', 'B-n', 'O'],
 ['卡', 'E-n', 'O'],
 ['模', 'B-n', 'O'],
 ['式', 'E-n', 'O'],
 ['，', 'S-x', 'O'],
 ['先', 'S-d', 'O'],
 ['通', 'B-p', 'O'],
 ['过', 'E-p', 'O'],
 ['跑', 'B-n', 'O'],
 ['马', 'E-n', 'O'],
 ['圈', 'B-n', 'O'],
 ['地', 'E-n', 'O']]

### 2. 构造CRF特征

![jupyter](./imgs/crf_features.png)


- 转移特征𝑡𝑘(𝑦𝑖−1,𝑦𝑖,𝑥,𝑖)是定义在边上的特征函数（transition），依赖于当前位置i和前一位置i-1，对应的权值为𝜆𝑘 

- 状态特征𝑠𝑙(𝑦𝑖,𝑥,𝑖)是定义在节点上的特征函数（state）依赖于当前位置i，对应的权值为𝜇𝑙  

- 特征函数的取值为1或0，当满足规定好的特征条件时取值为1，否则为0。


In [8]:
def word_features(sent, i):
    # 把第i个位置上面的特征拿出来
    word = sent[i][0]
    pos_tag = sent[i][1]

    features = {
        'bias': 1.0,
        # 是不是字母
        'word.isalpha()': word.encode('UTF-8').isalpha(),
        # 小写
        'word.lower()': word.lower(),
        # 大写
        'word.isupper()': word.isupper(),
        # 数字
        'word.isdigit()': word.isdigit(),
        # pos tag特征如B-v
        'pos_tag': pos_tag,
        # pos tag边界特征，如B I E S
        'pos_tag1': pos_tag.split('-')[0],
        # pos tag内容： v/n等
        'pos_tag2': pos_tag.split('-')[1],
    }
    # 如果不是开头，获取前一个位置的所有特征
    if i > 0:
        # 获取前一个字符
        word1 = sent[i - 1][0]
        # 获取前一个字符对应的pos tag
        pos_tag1 = sent[i - 1][1]
        # 将前一个字符的特征加入当前字符的特征
        features.update(
            {
                '-1:word.isalpha()': word1.encode('UTF-8').isalpha(),
                '-1:word.lower()': word1.lower(),
                '-1:word.isupper()': word1.isupper(),
                '-1:word.isdigit()': word1.isdigit(),
                '-1:pos_tag': pos_tag1,
                '-1:pos_tag1': pos_tag1.split('-')[0],
                '-1:pos_tag2': pos_tag1.split('-')[1],
            }
        )
    else:
        features['BOS'] = True  # 标记是开始beg of seq

    if i < len(sent) - 1:  # 如果不是最后一个，获取下一个位置的所有特征
        word2 = sent[i + 1][0]
        pos_tag2 = sent[i + 1][1]
        features.update(
            {
                '+1:word.isalpha()': word2.encode('UTF-8').isalpha(),
                '+1:word.lower()': word2.lower(),
                '+1:word.isupper()': word2.isupper(),
                '+1:word.isdigit()': word2.isdigit(),
                '+1:pos_tag': pos_tag2,
                '+1:pos_tag1': pos_tag2.split('-')[0],
                '+1:pos_tag2': pos_tag2.split('-')[1],
            }
        )
    else:
        features['EOS'] = True  # 标记是结束 end of seq

    return features


def sent_features(sent):
    return [word_features(sent, i) for i in range(len(sent))]


def sent_labels(sent):
    return [label for token, pos_tag, label in sent]


def sent_tokens(sent):
    return [token for token, pos_tag, label in sent]

In [6]:
# 特征抽取例子
sent_features(train_sentences[0])[1]

{'bias': 1.0,
 'word.isalpha()': False,
 'word.lower()': '商',
 'word.isupper()': False,
 'word.isdigit()': False,
 'pos_tag': 'E-nz',
 'pos_tag1': 'E',
 'pos_tag2': 'nz',
 '-1:word.isalpha()': False,
 '-1:word.lower()': '浙',
 '-1:word.isupper()': False,
 '-1:word.isdigit()': False,
 '-1:pos_tag': 'B-nz',
 '-1:pos_tag1': 'B',
 '-1:pos_tag2': 'nz',
 '+1:word.isalpha()': False,
 '+1:word.lower()': '银',
 '+1:word.isupper()': False,
 '+1:word.isdigit()': False,
 '+1:pos_tag': 'B-n',
 '+1:pos_tag1': 'B',
 '+1:pos_tag2': 'n'}

In [7]:
# 对bio数据进行特征抽取，以及标签提取
X_train = [sent_features(s) for s in train_sentences]
y_train = [sent_labels(s) for s in train_sentences]

X_test = [sent_features(s) for s in test_sentences]
y_test = [sent_labels(s) for s in test_sentences]

### 3. 使用sklearn-crfsuite进行训练
sklearn crfsuite是一个精简的crfsuite（python-crfsuite）包装器，  
它提供与scikit-learn兼容的sklearn_crfsuite.CRF estimator。  
以使用scikit-learn模型选择实用程序（交叉验证、超参数优化），或者使用joblib保存/加载CRF模型。  
https://sklearn-crfsuite.readthedocs.io/en/latest/api.html#module-sklearn_crfsuite

#### sklearn_crfsuite.CRF()常用参数  
- algorithm (str, optional (default='lbfgs')) ：训练算法。允许值：
    + 'lbfgs' - Gradient descent using the L-BFGS method  
    + 'l2sgd' - Stochastic Gradient Descent with L2 regularization term  
    + 'ap' - Averaged Perceptron  
    + 'pa' - Passive Aggressive (PA)   
    + 'arow' - Adaptive Regularization Of Weight Vector (AROW)  
- min_freq (float, optional (default=0)) ：特征出现频率的截止阈值。CRFsuite将忽略训练数据中出现频率不大于min\u freq的特征。默认值为no cut-off。同CRF++中的-f参数。
- all_possible_states (bool, optional (default=False)) ：指定 CRFsuite 是否生成在训练数据中甚至不出现的状态特征（即，负状态特征）。当为 True 时，CRFsuite 生成状态特征，这些特征将属性和标签之间的所有可能组合相关联。假设属性和标签的数量分别为 A 和 L，该函数将生成 (A * L) 个特征。 启用此功能可能会提高标记准确性，因为 CRF 模型可以学习到一个item预测的label与参考label不一致的情况。 但是，此功能也可能会增加特征数量并大大减慢训练过程。 默认情况下禁用此功能。
- all_possible_transitions (bool, optional (default=False)) ：指定CRFsuite是否生成训练数据中甚至没有出现的转移特征（即负转移特征）。如果为True，CRFsuite将生成关联所有可能标签对的转移特征，f(s', s, o=null)，其中s为t时刻的的标签（y），o为t时刻的上下文（x）。假设训练数据中的标签个数为L，该函数将生成（L*L）个转移特征。默认情况下禁用此功能。
- c1 (float, optional (default=0)) ：L1 正则化的系数。如果指定了非零值，CRFsuite 将切换到 Orthant-Wise Limited-memory Quasi-Newton (OWL-QN，Orthant-Wise 有限记忆拟牛顿) 方法。默认值为零（无 L1 正则化）。支持的训练算法：lbfgs。
- c2 (float, optional (default=1.0))：L2 正则化的系数。 支持的训练算法：l2sgd、lbfgs。
- max_iterations (int, optional (default=None)) ：优化算法的最大迭代次数。默认值取决于训练算法：
    + lbfgs - unlimited;  
    + l2sgd - 1000;   
    + ap - 100;  
    + pa - 100;  
    + arow - 100.  
- num_memories (int, optional (default=6)) ：用于逼近逆 Hessian 矩阵的有限内存数量。 支持的训练算法：lbfgs。
- epsilon (float, optional (default=1e-5))：确定收敛条件的 epsilon 参数。 支持的训练算法：ap、arow、lbfgs、pa
- period (int, optional (default=10)) ：测试停止标准的迭代持续时间。 支持的训练算法：l2sgd、lbfgs
- delta (float, optional (default=1e-5)) ：停止标准的阈值；当最后一个周期迭代中对数似然的改进不大于此阈值时，迭代停止。 支持的训练算法：l2sgd、lbfgs


In [8]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',  # 训练算法
    c1=0.0,  # l1正则
    c2=1.0,  # l2正则
    max_iterations=500,  # 最大迭代次数
    all_possible_transitions=True,  # 生成关联所有可能标签对的转移特征
    verbose=True  # 过程可视化
)

In [18]:
# 在训练集上训练
crf.fit(X_train, y_train)

loading training data to CRFsuite: 100%|██████████| 10748/10748 [00:04<00:00, 2398.14it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 49975
Seconds required: 1.742

L-BFGS optimization
c1: 0.000000
c2: 1.000000
num_memories: 6
max_iterations: 500
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=2.19  loss=695418.84 active=49975 feature_norm=1.00
Iter 2   time=2.03  loss=439787.83 active=49975 feature_norm=2.98
Iter 3   time=1.05  loss=387484.54 active=49975 feature_norm=2.59
Iter 4   time=2.10  loss=356803.40 active=49975 feature_norm=2.29
Iter 5   time=1.09  loss=345023.91 active=49975 feature_norm=2.32
Iter 6   time=1.06  loss=319775.35 active=49975 feature_norm=2.95
Iter 7   time=1.09  loss=291709.40 active=49975 feature_norm=3.85
Iter 8   time=1.09  loss=258508.09 active=49975 feature_norm=5.31
Iter 9   time=1.04  loss=231355.93 active=49975 feature_norm=7.25
Iter 

KeyboardInterrupt: 

In [10]:
import joblib

# 保存模型
model_path = 'crf/crf.model'
joblib.dump(crf, model_path)

['crf/crf.model']

### 4. 模型评估

In [11]:
# 获取所有labels
labels = list(crf.classes_)
# 去掉'O'标签，只关注实体的标签
labels.remove('O')
labels

['B-company',
 'I-company',
 'B-name',
 'I-name',
 'B-game',
 'I-game',
 'B-organization',
 'I-organization',
 'B-movie',
 'I-movie',
 'B-position',
 'I-position',
 'B-address',
 'I-address',
 'B-government',
 'I-government',
 'B-scene',
 'I-scene',
 'B-book',
 'I-book']

In [12]:
# 使用模型进行预测
y_pred = crf.predict(X_test)

In [13]:
# 将真实标签列表与预测标签列表展开成一个大的列表，进行结果评价
true_bio = [x for y in y_test for x in y]
predict_bio = [x for y in y_pred for x in y]

# 输出评价结果
metric_result = util.measure_by_tags(true_bio, predict_bio)


                precision    recall  f1-score   support

B-organization       0.52      0.39      0.44       206
     B-company       0.64      0.53      0.58       279
  B-government       0.73      0.60      0.66       190
        I-name       0.59      0.54      0.56       732
    B-position       0.69      0.53      0.60       347
        B-game       0.67      0.71      0.69       226
       B-scene       0.56      0.31      0.40       124
       I-scene       0.64      0.41      0.50       458
       I-movie       0.61      0.56      0.59       580
        I-book       0.67      0.54      0.60       715
     B-address       0.57      0.38      0.46       273
        I-game       0.69      0.74      0.71      1065
        B-name       0.67      0.56      0.61       352
       B-movie       0.58      0.47      0.52       101
     I-company       0.63      0.56      0.59      1031
  I-government       0.68      0.58      0.63       855
    I-position       0.67      0.54      0.60 

### 5. 模型预测

In [4]:
import joblib
# 加载模型

model_path = 'crf/crf.model'
saved_model = joblib.load(model_path)
saved_model

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.0, c2=1.0,
    keep_tempfiles=None, max_iterations=500, verbose=True)

In [5]:
# 定义CRF模型的批量预测pipeline，pos tag特征抽取 -> CRF特征函数抽取 -> 模型预测 
def predict_pipeline(model, sentences):
    features = []
    for sentence in sentences:
        posseg_list = util.get_word_posseg_feature(sentence)  # 提取pos tag BIOES特征
        sentence_with_posseg = [(sentence[index], posseg_list[index]) for index in range(len(sentence))]
        features.append(sent_features(sentence_with_posseg))  # 提取CRF特征
    y_predicts = model.predict(features)  # 使用模型做批量预测
    return y_predicts

In [9]:
test_sentences = ['李正茂出任中国电信集团有限公司总经理。',
                  '2012年成立中国电信国际有限公司,总部设于中国香港。',
                  '《长津湖》将于今年下半年上映。']

# 批量预测bio序列结果
y_predicts = predict_pipeline(saved_model, test_sentences)

In [10]:
for pred, sentence in zip(y_predicts, test_sentences):
    print(sentence)
    print(pred)
    # 将bio序列转化为实体序列
    entitys = util.bio_2_entities(pred)
    # 格式化展示实体序列结果
    result = util.formatting_result(entitys, sentence)
    print(result)

李正茂出任中国电信集团有限公司总经理。
['B-name', 'I-name', 'I-name', 'O', 'O', 'B-company', 'I-company', 'I-company', 'I-company', 'I-company', 'I-company', 'I-company', 'I-company', 'I-company', 'I-company', 'B-position', 'I-position', 'I-position', 'O']
[
    {
        "begin": 0,
        "end": 3,
        "tag": "name",
        "word": "李正茂"
    },
    {
        "begin": 5,
        "end": 15,
        "tag": "company",
        "word": "中国电信集团有限公司"
    },
    {
        "begin": 15,
        "end": 18,
        "tag": "position",
        "word": "总经理"
    }
]
2012年成立中国电信国际有限公司,总部设于中国香港。
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-company', 'I-company', 'I-company', 'I-company', 'I-company', 'I-company', 'I-company', 'I-company', 'I-company', 'I-company', 'O', 'O', 'O', 'O', 'O', 'B-address', 'I-address', 'I-address', 'I-address', 'O']
[
    {
        "begin": 7,
        "end": 17,
        "tag": "company",
        "word": "中国电信国际有限公司"
    },
    {
        "begin": 22,
        "end": 26,
        "tag": "address